In [1]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
doc = nlp('Dr. Alex Smith chaired first board meeting at Google')

In [4]:
doc

Dr. Alex Smith chaired first board meeting at Google

In [6]:
print([(ent.text, ent.label_) for ent in doc.ents])

[('Alex Smith', 'PERSON'), ('Google', 'ORG')]


In [7]:
def add_title(doc):
    new_ents = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ent.start != 0:
            prev_token = doc[ent.start-1]
            if prev_token.text in ('Dr', 'Dr.', 'Mr','Mr.'):
                new_ent = Span(doc, ent.start-1, ent.end, label = ent.label_)
                new_ents.append(new_ent)
            else:
                new_ents.append(ent)
    doc.ents = new_ents
    return doc

In [8]:
nlp.add_pipe(add_title, after='ner')

In [9]:
doc = nlp('Dr. Alex Smith chaired first board meeting at Google')

In [11]:
print([(ent.text, ent.label_) for ent in doc.ents])

[('Dr. Alex Smith', 'PERSON')]


### Use of POS and Dependency parsing

In [12]:
nlp = spacy.load('en_core_web_sm')

In [13]:
doc = nlp('Alex Smith was working at Google')

In [16]:
displacy.render(doc, style='dep', options = {'compact':True, 'distance':100})

In [82]:
def get_person_orgs(doc):
    person_entities = [ent for ent in doc.ents if ent.label_=="PERSON"]
    for ent in person_entities:
        head = ent.root.head
        if head.lemma_ == "work":
            preps = [token for token in head.children if token.dep_ == 'prep']
            for prep  in preps:
                orgs = [token for token in prep.children if token.ent_type_ == 'ORG']
                aux = [token for token in head.children if token.dep_=='aux']
                past_aux = any(t.tag_ == 'VBD' for t in aux)
                past = head.tag_ == 'VBD' or head.tag_ == 'VBG' and past_aux
                print({"person":ent, "orgs":orgs,"past": past})
    return doc

In [83]:
from spacy.pipeline import merge_entities

In [84]:
nlp = spacy.load('en_core_web_sm')

In [85]:
nlp.add_pipe(merge_entities)

In [86]:
nlp.add_pipe(get_person_orgs)

In [91]:
doc = nlp('Alex Smith worked at Google')

{'person': Alex Smith, 'orgs': [Google], 'past': True}
